# सेम्यान्टिक कर्नेल

यस कोड नमूनामा, तपाईं [सेम्यान्टिक कर्नेल](https://aka.ms/ai-agents-beginners/semantic-kernel) एआई फ्रेमवर्क प्रयोग गरेर एउटा आधारभूत एजेन्ट बनाउनुहुनेछ।

यस नमूनाको उद्देश्य तपाईंलाई ती चरणहरू देखाउनु हो जुन हामी पछि विभिन्न एजेन्टिक ढाँचाहरू कार्यान्वयन गर्दा थप कोड नमूनाहरूमा प्रयोग गर्नेछौं।


## आवश्यक Python प्याकेजहरू आयात गर्नुहोस्


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## क्लाइन्ट बनाउने

यस उदाहरणमा, हामी [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) प्रयोग गरेर LLM पहुँच गर्नेछौं।

`ai_model_id` लाई `gpt-4o-mini` भनेर परिभाषित गरिएको छ। GitHub Models बजारमा उपलब्ध अन्य मोडेलमा परिवर्तन गरेर विभिन्न नतिजा हेर्न प्रयास गर्नुहोस्।

हामीलाई `Azure Inference SDK` प्रयोग गर्नको लागि, जुन GitHub Models को `base_url` को लागि प्रयोग गरिन्छ, हामी Semantic Kernel भित्र `OpenAIChatCompletion` कनेक्टर प्रयोग गर्नेछौं। अन्य [उपलब्ध कनेक्टरहरू](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) पनि छन् जसले अन्य मोडेल प्रदायकहरूको लागि Semantic Kernel प्रयोग गर्न अनुमति दिन्छ।


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## एजेन्ट बनाउँदै 

तल हामी `TravelAgent` नामक एजेन्ट बनाउँछौं।

यस उदाहरणका लागि, हामी धेरै साधारण निर्देशनहरू प्रयोग गर्दैछौं। तपाईं यी निर्देशनहरू परिवर्तन गरेर हेर्न सक्नुहुन्छ कि एजेन्टले कसरी फरक प्रतिक्रिया दिन्छ।


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## एजेन्ट चलाउने

अब हामी `ChatHistoryAgentThread` प्रकारको थ्रेड परिभाषित गरेर एजेन्ट चलाउन सक्छौं। कुनै पनि आवश्यक प्रणाली सन्देशहरू एजेन्टको invoke_stream को `messages` कुञ्जीवर्ड आर्गुमेन्टमा प्रदान गरिन्छ।

यी परिभाषित भएपछि, हामी `user_inputs` सिर्जना गर्छौं, जुन प्रयोगकर्ताले एजेन्टलाई पठाउने सन्देश हो। यस अवस्थामा, हामीले यो सन्देश `Plan me a sunny vacation` मा सेट गरेका छौं।

यो सन्देश परिवर्तन गर्न स्वतन्त्र महसुस गर्नुहोस् र हेर्नुहोस् कि एजेन्टले कसरी फरक प्रतिक्रिया दिन्छ।


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी यथासम्भव सटीकता सुनिश्चित गर्न प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादमा त्रुटिहरू वा अशुद्धताहरू हुन सक्छन्। यसको मूल भाषामा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्त्वपूर्ण जानकारीका लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याको लागि हामी जिम्मेवार हुने छैनौं।  
